In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import scipy.linalg

from graph_utils import graph_gen, find_alpha_lower_bound
from problem import Synthetic
from plot_utils import MyFigure

from algorithms import IPLUX, UDC, dual_subgradient
# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')


In [ ]:
# generate problem

# network

num_node = 20
num_edge = 26

# np.random.seed(1)

graph_gen(num_node, num_edge, B=1)

net_dir = f'data/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)
# print(network)

# problem data

parameters = {}
parameters['N'] = num_node
parameters['d'] = 3 # dimension of x_i's
parameters['m'] = 1 # number jof inequality constraints
parameters['p'] = 5 # number of equality constraints

prob = Synthetic(parameters)
# prob = Synthetic(parameters, debug=True)

prob.gen()
prob.load()
alpha_lower = find_alpha_lower_bound(parameters, prob)
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')

In [5]:
# load instance 
instance_name = 'instance1_N20E40'
# instance_name = 'instance2_N20E26'
instance_dir = 'instance/' + instance_name

# network
network = np.load(instance_dir + '/graph/subgraph_W.npy')
num_node = network.shape[0]
print(network.shape)

# problem data
parameters = {}
parameters['N'] = num_node
parameters['d'] = 1 # temporary, reset in load()
parameters['m'] = 1 # temporary
parameters['p'] = 1 # temporary

prob = Synthetic(parameters)
prob.save_dir = instance_dir + '/problem'
prob.load()

alpha_lower = find_alpha_lower_bound(parameters, prob)
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')


(20, 20)
loading a Synthetic problem, N=20
problem loaded:
Q: (20, 3), P: (20, 3, 3)
A: (20, 5, 3)
a: (20, 3), c: (20,)
aa: (20, 3), ca: (20,)
x_star (20, 3)
opt_val -1.7937811849951981
x* [[ 1.69192667e-08 -4.54451916e-02 -1.25326407e-02]
 [-1.97438972e-11  2.94708451e-10  1.96457124e-01]
 [-1.55498077e-01 -1.81713964e-10 -4.30739243e-02]
 [ 2.21417334e-10  6.41377197e-11 -2.57323583e-01]
 [ 5.46491556e-01 -3.09078549e-01 -2.25841238e-10]
 [-7.95790832e-10  4.77051274e-01 -9.77296145e-02]
 [ 2.30311170e-01  9.67073127e-10 -4.08278630e-02]
 [ 4.41710887e-10  5.03547130e-01  2.55567666e-02]
 [-4.78912681e-01 -1.55547314e-09  3.47456666e-10]
 [ 1.16595532e-10 -9.51339774e-11 -4.60428545e-01]
 [ 4.42020248e-10 -4.19241406e-01  1.76861413e-01]
 [-1.46017592e-09  6.03865377e-10 -1.04286398e-09]
 [-3.70053317e-01  6.45150319e-01  3.92380391e-10]
 [ 1.95357925e-10 -2.86013131e-10 -4.44910108e-10]
 [-1.46178792e-08 -3.47711897e-01  8.41473423e-10]
 [-8.31629811e-10 -1.66205909e-01  1.02254197e

In [65]:
# IPLUX
# Instance 1: a5.0 r5
# Instance 2: a20_r4 130.59s

MAX_ITER = 2000
# MAX_ITER = 500

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

alpha_lower = find_alpha_lower_bound(parameters, prob)
# alpha_choice = [alpha_lower, alpha_lower*5, alpha_lower*25]
alpha_choice = [alpha_lower, alpha_lower+1]
# alpha_choice = [alpha_lower+1]

# rho_choice = [1, 5, 25, 0.5]
rho_choice = [0.5, 1, 5, 10]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')

np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})
for alpha in alpha_choice:
    for rho in rho_choice:
        
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        # if True:
        # if rho != 5:
        if rho == 10 and alpha == 5.0:
            alg = IPLUX(prob, network, alpha=alpha, rho=rho, verbose=False)
            for i in range(MAX_ITER):
                alg.step()
        
        prefix = f'{alg.name}_a{alpha}_r{rho}'
        filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
        filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
        filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
        
        obj_err_figure.add_line_file(prefix, filename_oe, style=c+ls)
        cons_vio_figure.add_line_file(prefix, filename_cv, style=c+ls)
        x_dis_figure.add_line_file(prefix, filename_xd, style=c+ls)
            
        obj_err_figure.paint(MAX_ITER=MAX_ITER)
        cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
        x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

self.prob minimize param21986 @ var21985 + norm1(var21985) + 2.5 @ QuadForm(var21985 + -param21987, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) + 0.05 @ quad_over_lin(param21988 @ var21985, 1.0) + param21989 @ var21985 + param21990[0] @ QuadForm(var21985, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) + Promote(-2.0, (3,)) @ param21991 @ var21985
subject to QuadForm(var21985 + -param21993, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param21994[0]
reset
IPLUX alpha 5.0 rho 10, iter 0, obj err: 1.79e+00, cons vio: 0.00e+00
time 0.01, saved

IPLUX alpha 5.0 rho 10, iter 100, obj err: 2.89e-01, cons vio: 5.57e-01
time 5.50, saved

IPLUX alpha 5.0 rho 10, iter 200, obj err: 1.19e-01, cons vio: 2.61e-01
time 10.95, saved

IPLUX alpha 5.0 rho 10, iter 300, obj err: 2.91e-02, cons vio: 1.17e-01
time 16.30, saved

IPLUX alpha 5.0 rho 10, iter 400, obj err: 2.55e-03, cons vio: 3.34e-02
time 21.58, saved

IPLUX alpha 5.0 rho 10, iter 500, obj err: 8.43e-03, con

In [49]:
# UDC

# best parameters:

# instance 1
# UDC_PEXTRA: r4
# UDC_PGC: r0.4
# UDC_DPGA: r2
# UDC_DistADMM: r1
# ALT: r2

# instance 2
# UDC_PEXTRA: a0_r1
# UDC_pt: a20_r4, 156.29s
# UDC_DistADMM: a0_r2


# MAX_ITER = 500
MAX_ITER = 2000

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# rho_choice = [0.1, 1, 10]
# rho_choice = [0.5, 1, 2, 4]
# rho_choice = [0.05, 0.1, 0.2, 0.4, 0.8]
rho_choice = [2]

# alpha_choice = [0, 10, 100]
# alpha_choice = [0, 0.1]
alpha_choice = [0]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')
    
np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})
for alpha in alpha_choice:
    for rho in rho_choice:
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        if True:
        # if alpha != 0:
        # if rho != 1:
        # if rho == 4:
            # alg = UDC(prob, network, rho=rho, alpha=alpha,
            #           param_setting='PEXTRA', verbose=False)
            # alg = UDC(prob, network, rho=rho, alpha=alpha,
            #           param_setting='PGC', verbose=False)
            # alg = UDC(prob, network, rho=rho, alpha=alpha,
            #           param_setting='DPGA', verbose=False)
            # alg = UDC(prob, network, rho=rho, alpha=alpha,
            #           param_setting='DistADMM', verbose=False)
            alg = UDC(prob, network, rho=rho, alpha=alpha,
                    param_setting='ALT', verbose=False)
            for i in range(MAX_ITER):
                alg.step()
            
        prefix = f'{alg.name}_a{alpha}_r{rho}'
        filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
        filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
        filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
        
        obj_err_figure.add_line_file(prefix, filename_oe, style=c+ls)
        cons_vio_figure.add_line_file(prefix, filename_cv, style=c+ls)
        x_dis_figure.add_line_file(prefix, filename_xd, style=c+ls)
            
        obj_err_figure.paint(MAX_ITER=MAX_ITER)
        cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
        x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


UDC setting: ALT
self.prob minimize quad_over_lin(param11489 @ var11488, 1.0) + param11490 @ var11488 + param11491[0] @ norm1(var11488) + 0.0 @ quad_over_lin(Promote(param11492[0], (3,)) @ var11488 + -param11493, 1.0) + 0.5 @ quad_over_lin(maximum(param11494 + quad_over_lin(var11488 + -param11496, 1.0) + -param11497, 0.0), 1.0) + 0.5 @ quad_over_lin(param11495 + param11500 @ var11488, 1.0)
subject to QuadForm(var11488 + -param11498, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param11499[0]
reset
ALT alpha 0 rho 2, iter 0, obj err: 1.79e+00, cons vio: 0.00e+00
time 0.01, saved

ALT alpha 0 rho 2, iter 100, obj err: 2.51e-02, cons vio: 6.61e-02
time 6.27, saved

ALT alpha 0 rho 2, iter 200, obj err: 1.69e-03, cons vio: 3.24e-03
time 12.57, saved

ALT alpha 0 rho 2, iter 300, obj err: 3.30e-04, cons vio: 1.63e-03
time 18.62, saved

ALT alpha 0 rho 2, iter 400, obj err: 6.24e-05, cons vio: 3.35e-04
time 24.43, saved

ALT alpha 0 rho 2, iter 500, obj err: 1.46e-06, cons vio: 

In [ ]:
H2 = 0.5 * (np.identity(num_node)-network)
np.linalg.eigvals(H2)
# scipy.linalg.sqrtm(H2)

In [6]:
# DPMM
# instance 1: a0.1_r5_t1.5
# instance 2: a0.1_r1_t1.5

from plot_utils import MyFigure

alg_name = 'DPMM'
MAX_ITER = 2000
# MAX_ITER = 500

# theta_choice = [0.5, 1, 1.5, 1.9]
# theta_choice = [0.1, 1.9]
theta_choice = [1.5]

# rho_choice = [0.1, 1, 10]
# rho_choice = [0.5, 1, 2, 4, 5]
rho_choice = [5]

# alpha_choice = [0, 10, 100]
# alpha_choice = [0.1, 1, 10]
alpha_choice = [0.1]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')


for alpha in alpha_choice:
    for rho in rho_choice:
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        for theta in theta_choice:
            
            if False:
            # if rho != 1:
            # if rho == 5:
            # if alpha != 0:
            # if theta == 1.9:
                alg = UDC(prob, network, rho=rho, alpha=alpha, theta=theta,
                            param_setting='DPMM', verbose=False)
                for i in range(MAX_ITER):
                    alg.step()

            # plot
            # ci = theta_choice.index(theta)
            # c = color[ci]
            
            prefix = f'{alg_name}_a{alpha}_r{rho}_t{theta}'
            filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
            filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
            filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
            
            obj_err_figure.add_line_file(prefix, filename_oe, style=c+ls)
            cons_vio_figure.add_line_file(prefix, filename_cv, style=c+ls)
            x_dis_figure.add_line_file(prefix, filename_xd, style=c+ls)
                
            obj_err_figure.paint(MAX_ITER=MAX_ITER)
            cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
            x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


c:\Users\xtq\OneDrive - shanghaitech.edu.cn\Research\Dual AMM\numerical\exp3_synthetic_IPLUX\plot_utils.py:131: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  fig.savefig(f'{self.filename}.png', bbox_inches='tight', transparent=False)


In [ ]:
# dual subgradient
#
# instance 1: g5
# instance 2: g10

from plot_utils import MyFigure

alg_name = 'dual_subgradient'
MAX_ITER = 500
# gamma_choice = [0.1, 1, 5, 25]
# gamma_choice = [2.5, 5, 10, 50]
gamma_choice = [2.5, 5, 10]
color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')


for gamma in gamma_choice:
    if True:
    # if gamma == 50:
        alg = dual_subgradient(prob, network, gamma)
        for i in range(MAX_ITER):
            alg.step()

    # plot
    ci = gamma_choice.index(gamma)
    c = color[ci]
    
    prefix = f'{alg_name}_g{gamma}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    obj_err_figure.add_line_file(prefix, filename_oe, style=c)
    cons_vio_figure.add_line_file(prefix, filename_cv, style=c)
    x_dis_figure.add_line_file(prefix, filename_xd, style=c)
        
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

    

In [50]:
# compare fine-tuned algorithms

# instance 1
# UDC_PEXTRA: r4
# UDC_PGC: r0.4
# UDC_DPGA: r2
# UDC_DistADMM: r1
# ALT: r2
# DPMM: a0.1 r5 t1.5
# IPLUX: a5.0 r5
# dual subgradient: g5

alg_list = []
alg_list.append(UDC(prob, network, alpha=0, rho=4, param_setting='PEXTRA'))
alg_list.append(UDC(prob, network, alpha=0, rho=0.4, param_setting='PGC'))
alg_list.append(UDC(prob, network, alpha=0, rho=2, param_setting='DPGA'))
alg_list.append(UDC(prob, network, alpha=0, rho=1, param_setting='DistADMM'))
alg_list.append(UDC(prob, network, alpha=0, rho=2, param_setting='ALT'))
alg_list.append(UDC(prob, network, alpha=0.1, rho=5, theta=1.5, 
                                                    param_setting='DPMM'))
alg_list.append(IPLUX(prob, network, alpha=5.0, rho=5))
alg_list.append(dual_subgradient(prob, network, gamma=5))
oe_logs = dict()
cv_logs = dict()
xd_logs = dict()


# instance 2
# param_best_list = [['IPLUX', 4.0, 1], 
#                  ['UDC_pt', 20, 4], 
#                  ['UDC_PEXTRA', 0.1, 1], 
#                  ['UDC_DistADMM', 0.1, 2],
#                  ['DPMM', 0.1, 1, 1.5],
#                  ['dual_subgradient', 10]]
# alg_list = []
# alg_list.append(IPLUX(prob, network, alpha=4.0, rho=1))
# alg_list.append(UDC(prob, network, alpha=20, rho=4, param_setting='pt'))
# alg_list.append(UDC(prob, network, alpha=0.1, rho=1, param_setting='PEXTRA'))
# alg_list.append(UDC(prob, network, alpha=0.1, rho=2, param_setting='DistADMM'))
# alg_list.append(UDC(prob, network, alpha=0.1, rho=1, theta=1.5, param_setting='DPMM'))
# alg_list.append(dual_subgradient(prob, network, gamma=10))




color = ['tab:green', 'chocolate', 'red', 'chartreuse', 'navy', 'grey', 'k', 'crimson']
marker = ['.', 'o', 'v', '^', '8', 's', '*', 'd', 'X']
MAX_ITER = 2000

obj_err_figure = MyFigure(filename='compare_obj_err', 
                            xlabel=r'$\mathrm{Iteration}$ $k$', 
                            ylabel=r'$\mathrm{Objective}\ \mathrm{Error}$')

cons_vio_figure = MyFigure(filename='compare_cons_vio', 
                            xlabel=r'$\mathrm{Iteration}$ $k$', 
                            ylabel=r'$\mathrm{Constraint}\ \mathrm{Violation}$')

x_dis_figure = MyFigure(filename='compare_x_distance', 
                            xlabel=r'$\mathrm{Iteration}$ $k$', 
                            ylabel=r'$\Vert \mathbf{x}^k - \mathbf{x}^\star \Vert$')

for alg in alg_list:
    ci = alg_list.index(alg)
    c = color[ci]
    m = marker[ci]
    # for i in range(MAX_ITER):
    #     alg.step()
    # log_dir = 'log/N20'
    # filename_oe = f'{log_dir}/{alg.file_prefix}_oe.txt'
    # filename_cv = f'{log_dir}/{alg.file_prefix}_cv.txt'
    # filename_xd = f'{log_dir}/{alg.file_prefix}_xd.txt'
    
    # instance 1
    log_dir = f'instance/instance1_N20E40/log'
    file_alg_name = alg.name.replace(' ', '_').replace('-', '_')
    filename_oe = f'{log_dir}/{file_alg_name}/all_log/{alg.file_prefix}_oe.txt'
    filename_cv = f'{log_dir}/{file_alg_name}/all_log/{alg.file_prefix}_cv.txt'
    filename_xd = f'{log_dir}/{file_alg_name}/all_log/{alg.file_prefix}_xd.txt'
    oe_logs[alg.name] = np.loadtxt(filename_oe)
    cv_logs[alg.name] = np.loadtxt(filename_cv)
    xd_logs[alg.name] = np.loadtxt(filename_xd)
    
    # obj_err_figure.add_line_file(alg.name, filename_oe, style=c, marker=m)
    # cons_vio_figure.add_line_file(alg.name, filename_cv, style=c, marker=m)
    # x_dis_figure.add_line_file(alg.name, filename_xd, style=c, marker=m)

    # obj_err_figure.paint(MAX_ITER=MAX_ITER)
    # cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    # x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


UDC setting: PEXTRA
self.prob minimize quad_over_lin(param1429 @ var1428, 1.0) + param1430 @ var1428 + param1431[0] @ norm1(var1428) + 0.0 @ quad_over_lin(Promote(param1432[0], (3,)) @ var1428 + -param1433, 1.0) + 0.5 @ quad_over_lin(maximum(param1434 + quad_over_lin(var1428 + -param1436, 1.0) + -param1437, 0.0), 1.0) + 0.5 @ quad_over_lin(param1435 + param1440 @ var1428, 1.0)
subject to QuadForm(var1428 + -param1438, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param1439[0]
reset
UDC_PEXTRA alpha 0 rho 4, iter 0, obj err: 1.79e+00, cons vio: 0.00e+00
time 0.02, saved

UDC setting: PGC
self.prob minimize quad_over_lin(param1481 @ var1480, 1.0) + param1482 @ var1480 + param1483[0] @ norm1(var1480) + 0.0 @ quad_over_lin(Promote(param1484[0], (3,)) @ var1480 + -param1485, 1.0) + 0.5 @ quad_over_lin(maximum(param1486 + quad_over_lin(var1480 + -param1488, 1.0) + -param1489, 0.0), 1.0) + 0.5 @ quad_over_lin(param1487 + param1492 @ var1480, 1.0)
subject to QuadForm(var1480 + -pa

In [90]:

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
from matplotlib.ticker import FormatStrFormatter, LogFormatter
from matplotlib.legend_handler import HandlerTuple
# rc('mathtext', fontset='stixsans')
# rc('text', usetex=True)
# rc('text', usetex=True)
# rc('mathtext', fontset ='dejavusans')
# rc('font', **{'family': 'sans-serif', 'sans-serif': ['DejaVu Sans'], 'size': 40})

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.serif": "Computer Modern Roman",
    "font.sans-serif": "Helvetica",
    "font.size": 42,
    'text.latex.preamble': r'\usepackage{sfmath}' # sans-serif for math
})

fig, ax = plt.subplots(figsize=(20, 12))   
ax.set_xlim(0, MAX_ITER)
ax.set_yscale('log')
ax.set_xlabel(r'Iteration $k$', fontsize=42)
ax.set_ylabel(r'Objective Error', fontsize=42)
ax.tick_params(axis='both', labelsize=42)
ax.yaxis.set_minor_locator(plt.LogLocator())
ax.spines.bottom.set_linewidth(2)
ax.spines.top.set_linewidth(2)
ax.spines.left.set_linewidth(2)
ax.spines.right.set_linewidth(2)
plt.grid(True, ls="--", color='0.65')


p1, = ax.plot(oe_logs['UDC_PEXTRA'], label='UDC-SingleRound', lw=3, c='xkcd:blue')
p2, = ax.plot(oe_logs['UDC_PGC'], lw=3, c='#00BFFF', alpha=0.5) # , label='UDC-PGC' sky blue
p3, = ax.plot(oe_logs['UDC_DPGA'], lw=3, c='#6699CC', alpha=0.4) # , label='UDC-DPGA' blue gray
p4, = ax.plot(oe_logs['UDC_DistADMM'], label='UDC-Dist.ADMM', ls='-', c='xkcd:green',lw=3)
p5, = ax.plot(oe_logs['ALT'], label='ALT', ls='-', lw=3, c='xkcd:orangered')
p6, = ax.plot(oe_logs['DPMM'], label='DPMM', ls='-.', c='orange', lw=3)
p7, = ax.plot(oe_logs['IPLUX'], label='IPLUX', ls='-.', c='xkcd:purple', lw=3)
p8, = ax.plot(oe_logs['dual subgradient'], label='dual subgradient', ls='-.', c='olive', lw=3)

ax.legend([(p1, p2, p3), p4, p5, p6, p7, p8], 
          ['UDC-SingleRound', 'UDC_DistADMM', 'ALT', 'DPMM', 
           'IPLUX', 'dual subgradient'], 
            handler_map={tuple: HandlerTuple(ndivide=None)},
            fontsize=40)
# ax.legend(fontsize=40)
fig.savefig(f'compare_obj_err.pdf', bbox_inches='tight', transparent=False)




fig, ax = plt.subplots(figsize=(20, 12))   
ax.set_xlim(0, MAX_ITER)
ax.set_yscale('log')
ax.set_xlabel(r'Iteration $k$', fontsize=42)
ax.set_ylabel(r'Constraint Violation', fontsize=42)
ax.tick_params(axis='both', labelsize=42)
ax.yaxis.set_minor_locator(plt.LogLocator())
ax.spines.bottom.set_linewidth(2)
ax.spines.top.set_linewidth(2)
ax.spines.left.set_linewidth(2)
ax.spines.right.set_linewidth(2)
plt.grid(True, ls="--", color='0.65')


p1, = ax.plot(cv_logs['UDC_PEXTRA'], label='UDC-SingleRound', lw=3, c='xkcd:blue')
p2, = ax.plot(cv_logs['UDC_PGC'], lw=3, c='#00BFFF', alpha=0.5) # , label='UDC-PGC'
p3, = ax.plot(cv_logs['UDC_DPGA'], lw=3, c='#6699CC', alpha=0.4) # , label='UDC-DPGA'
p4, = ax.plot(cv_logs['UDC_DistADMM'], label='UDC-Dist.ADMM', ls='-', c='xkcd:green',lw=3)
p5, = ax.plot(cv_logs['ALT'], label='ALT', ls='-', lw=3, c='xkcd:orangered')
p6, = ax.plot(cv_logs['DPMM'], label='DPMM', ls='-.', c='orange', lw=3)
p7, = ax.plot(cv_logs['IPLUX'], label='IPLUX', ls='-.', c='xkcd:purple', lw=3)
p8, = ax.plot(cv_logs['dual subgradient'], label='dual subgradient', ls='-.', c='olive', lw=3)

ax.legend([(p1, p2, p3), p4, p5, p6, p7, p8], 
          ['UDC-SingleRound', 'UDC_DistADMM', 'ALT', 'DPMM', 
           'IPLUX', 'dual subgradient'], 
            handler_map={tuple: HandlerTuple(ndivide=None)},
            fontsize=40)
fig.savefig(f'compare_cons_vio.pdf', bbox_inches='tight', transparent=False)




fig, ax = plt.subplots(figsize=(20, 12))   
ax.set_xlim(0, MAX_ITER)
ax.set_yscale('log')
ax.set_xlabel(r'Iteration $k$', fontsize=42)
ax.set_ylabel(r'$\Vert\mathit{x^k}-\mathit{x}^\star\Vert$', fontsize=42)
ax.tick_params(axis='both', labelsize=42)
ax.yaxis.set_minor_locator(plt.LogLocator())
ax.spines.bottom.set_linewidth(2)
ax.spines.top.set_linewidth(2)
ax.spines.left.set_linewidth(2)
ax.spines.right.set_linewidth(2)
plt.grid(True, ls="--", color='0.65')


ax.plot(xd_logs['UDC_PEXTRA'], label='UDC-PEXTRA', lw=3, c='orange')
# ax.plot(xd_logs['UDC_PGC'], label='UDC-PGC', ls='--', lw=3)
# ax.plot(xd_logs['UDC_DPGA'], label='UDC-DPGA', ls='--', lw=3)
ax.plot(xd_logs['UDC_DistADMM'], label='UDC-Dist.ADMM', ls='-', c='xkcd:green',lw=3)
ax.plot(xd_logs['ALT'], label='ALT', ls='-', lw=3, c='xkcd:orangered')
ax.plot(xd_logs['DPMM'], label='DPMM', ls='-.', c='xkcd:blue', lw=3)
ax.plot(xd_logs['IPLUX'], label='IPLUX', ls='-.', c='xkcd:purple', lw=3)
ax.plot(xd_logs['dual subgradient'], label='dual subgradient', ls='-.', c='olive', lw=3)

ax.legend(fontsize=40)
fig.savefig(f'compare_x_distance.pdf', bbox_inches='tight', transparent=False)

In [ ]:

# ============================= trace in x1 x1 plane =========================
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig, ax = plt.subplots()     # Create a figure containing a single Axes.

delta = 0.05
x = np.arange(0.0, 1, delta)
y = np.arange(0.0, 1, delta)
X, Y = np.meshgrid(x, y)
F = X*alg.c[0] + Y*alg.c[1]
G = -alg.d[0]*np.log(1+X) - alg.d[1]*np.log(1+Y) + alg.b[0]*2/num_node

im = ax.imshow(F, interpolation='bilinear', origin='lower',
            cmap=cm.gray, extent=(0, 1, 0, 1))
CS_F = ax.contour(X, Y, F, 5)
ax.clabel(CS_F, fontsize=50)

# if there are only two nodes, we can polt the constraint
if num_node == 2:
    CS_G = ax.contour(X, Y, G, colors='k')
    ax.clabel(CS_G, fontsize=20)

x_log = np.array(alg.x_log)
x_avg_log = np.array(alg.x_avg_log)
ax.scatter(alg.x_star[0], alg.x_star[1], s=5000, c='r', marker='X', label='optimal')
ax.scatter(x_log[-1,0], x_log[-1,1], s=500, c='y', marker='^', label='last')
ax.scatter(x_log[:,0], x_log[:,1], label='iter', s=100, c='b', marker='^')  # Plot some data on the Axes.
ax.scatter(x_avg_log[:,0], x_avg_log[:,1], label='iter avg', s=100, marker='^')

ax.set_title('x position')
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.legend()
fig.savefig('x.png')
plt.close()
